In [ ]:
# packages
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import classification_report, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/lagged_curtailment_target_features.csv', sep = ';', index_col=0)

In [ ]:
df.index = pd.to_datetime(df.index)


In [ ]:
# Identify significant lag values (where autocorrelation is outside the confidence intervals)
significant_lags = [48, 96]

df_lagged = df.copy()

# Create lagged features
for lag in significant_lags:
    df_lagged[f'redispatch_lag_{lag}'] = df_lagged['redispatch'].shift(lag)
    df_lagged[f'level_lag_{lag}'] = df_lagged['level'].shift(lag)

# Drop rows with NaN values resulting from the shifting
df_lagged.dropna(inplace=True)

In [ ]:
df_lagged.columns

Index(['redispatch', 'level', 'wind_speed_m/s', 'wind_speed_m/s_lag1',
       'wind_speed_m/s_lag2', 'wind_speed_m/s_lag3', 'wind_speed_m/s_lag4',
       'wind_speed_m/s_lag5', 'wind_direction_degrees',
       'wind_direction_degrees_lag1', 'wind_direction_degrees_lag2',
       'wind_direction_degrees_lag3', 'wind_direction_degrees_lag4',
       'wind_direction_degrees_lag5', 'radiation_global_J/m2',
       'radiation_global_J/m2_lag1', 'radiation_global_J/m2_lag2',
       'radiation_global_J/m2_lag3', 'radiation_global_J/m2_lag4',
       'radiation_global_J/m2_lag5', 'air_temperature_K',
       'air_temperature_K_lag1', 'air_temperature_K_lag2',
       'air_temperature_K_lag3', 'air_temperature_K_lag4',
       'air_temperature_K_lag5', 'humidity_percent', 'humidity_percent_lag1',
       'humidity_percent_lag2', 'humidity_percent_lag3',
       'humidity_percent_lag4', 'humidity_percent_lag5', 'wind_gust_max_m/s',
       'wind_gust_max_m/s_lag1', 'wind_gust_max_m/s_lag2',
       'wind_g

In [ ]:
# get desired df size
start_date = '2022-01-01'
end_date = '2023-06-30'
df_lagged = df_lagged.loc[start_date:end_date]

In [ ]:
# Define features and target
features = df_lagged.columns
target = 'redispatch'

X = df_lagged[features].drop(['redispatch', 'level'], axis=1)
y = df_lagged[target]

# Define hyperparameters
params = {
    'max_depth': 20,
    'min_samples_split': 5,
    'min_samples_leaf': 5,
    'max_features': 'log2',
    'n_estimators': 500,
    'random_state': 42,
    'class_weight': 'balanced',
    'bootstrap': True
}

# Define the number of splits for time series cross-validation
n_splits = 10  # You can adjust this value as needed
gap = 48  # 12 hour difference between train and test sets

# Initialize lists to store evaluation metrics for each fold
train_f1_scores = []
train_precision_scores = []
train_recall_scores = []
test_f1_scores = []
test_precision_scores = []
test_recall_scores = []
thresh_f1_scores = []
thresh_precision_scores = []
thresh_recall_scores = []

# Define the time series splitter
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

# Define threshold for precision calculation
threshold = 0.45

# Initialize SMOTE
smote = SMOTE(k_neighbors=1, random_state=42)

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
    print(f"Training on fold {fold}/{n_splits}")
    # Get the data for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to balance the classes
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

    # Initialize and train the Extra Trees classifier
    model = ExtraTreesClassifier(**params)
    model.fit(X_train_balanced, y_train_balanced)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate the model using precision, recall, and F1 score
    train_f1 = f1_score(y_train, y_train_pred, zero_division=1)
    train_precision = precision_score(y_train, y_train_pred, zero_division=1)
    train_recall = recall_score(y_train, y_train_pred, zero_division=1)
    test_f1 = f1_score(y_test, y_test_pred, zero_division=1)
    test_precision = precision_score(y_test, y_test_pred, zero_division=1)
    test_recall = recall_score(y_test, y_test_pred, zero_division=1)

    # Compute precision, recall, and F1 score using the specified threshold
    y_test_proba = model.predict_proba(X_test)[:, 1]
    y_test_pred_threshold = (y_test_proba > threshold).astype(int)
    fold_precision = precision_score(y_test, y_test_pred_threshold, zero_division=1)
    fold_recall = recall_score(y_test, y_test_pred_threshold, zero_division=1)
    fold_f1 = f1_score(y_test, y_test_pred_threshold, zero_division=1)

    cm = confusion_matrix(y_test, y_test_pred_threshold)
    # Print the confusion matrix
    print(f"Confusion Matrix (Fold {fold}):")
    print(cm)
    print("\n")

    # Store the evaluation metrics for this fold
    train_f1_scores.append(train_f1)
    train_precision_scores.append(train_precision)
    train_recall_scores.append(train_recall)
    test_f1_scores.append(test_f1)
    test_precision_scores.append(test_precision)
    test_recall_scores.append(test_recall)
    thresh_f1_scores.append(fold_f1)
    thresh_precision_scores.append(fold_precision)
    thresh_recall_scores.append(fold_recall)

# Calculate average evaluation metrics across all folds
avg_train_f1 = np.mean(train_f1_scores)
avg_train_precision = np.mean(train_precision_scores)
avg_train_recall = np.mean(train_recall_scores)
avg_test_f1 = np.mean(test_f1_scores)
avg_test_precision = np.mean(test_precision_scores)
avg_test_recall = np.mean(test_recall_scores)
avg_thresh_f1_scores = np.mean(thresh_f1_scores)
avg_thresh_precision_scores = np.mean(thresh_precision_scores)
avg_thresh_recall_scores = np.mean(thresh_recall_scores)

# Print the results
print("Average Train F1 Score:", avg_train_f1)
print("Average Train Precision:", avg_train_precision)
print("Average Train Recall:", avg_train_recall)
print("\nAverage Test F1 Score:", avg_test_f1)
print("Average Test Precision:", avg_test_precision)
print("Average Test Recall:", avg_test_recall)
print("\nAverage Threshold F1 Score:", avg_thresh_f1_scores)
print("Average Threshold Precision:", avg_thresh_precision_scores)
print("Average Threshold Recall:", avg_thresh_recall_scores)

Training on fold 1/10
Confusion Matrix (Fold 1):
[[4643    0]
 [ 123    0]]


Training on fold 2/10
Confusion Matrix (Fold 2):
[[4218    0]
 [ 548    0]]


Training on fold 3/10
Confusion Matrix (Fold 3):
[[3422  342]
 [ 426  576]]


Training on fold 4/10
Confusion Matrix (Fold 4):
[[3481  355]
 [ 534  396]]


Training on fold 5/10
Confusion Matrix (Fold 5):
[[2696  837]
 [ 417  816]]


Training on fold 6/10
Confusion Matrix (Fold 6):
[[2446  654]
 [ 700  966]]


Training on fold 7/10
Confusion Matrix (Fold 7):
[[3698  482]
 [ 413  173]]


Training on fold 8/10
Confusion Matrix (Fold 8):
[[3831  392]
 [ 338  205]]


Training on fold 9/10
Confusion Matrix (Fold 9):
[[3864  501]
 [ 164  237]]


Training on fold 10/10
Confusion Matrix (Fold 10):
[[3795  835]
 [  36  100]]


Average Train F1 Score: 0.8984404642601109
Average Train Precision: 0.8286759337287473
Average Train Recall: 0.9859913589371727

Average Test F1 Score: 0.33394916883707887
Average Test Precision: 0.5380688072027373
Ave

Average Train F1 Score: 0.8984404642601109
Average Train Precision: 0.8286759337287473
Average Train Recall: 0.9859913589371727

Average Test F1 Score: 0.33394916883707887
Average Test Precision: 0.5380688072027373
Average Test Recall: 0.3773095099022651

Average Threshold F1 Score: 0.3465937986520303
Average Threshold Precision: 0.5280287932024957
Average Threshold Recall: 0.4241359803306919

1. with all features and for all years

Average Train F1 Score: 0.8919136140043056
Average Train Precision: 0.8125640769436469
Average Train Recall: 0.9946395181135141

 Average Test F1 Score: 0.14931013647497063
Average Test Precision: 0.2704487542166848
Average Test Recall: 0.14845568670700726

 Average Threshold F1 Score: 0.14503863955501922
Average Threshold Precision: 0.4748371186409834
Average Threshold Recall: 0.13406205628745235

2. without wind speed and for all years

Average Train F1 Score: 0.9101319063370739
Average Train Precision: 0.8411824844930612
Average Train Recall: 0.9963448322180584

 Average Test F1 Score: 0.14035396058573552
Average Test Precision: 0.3720906604082431
Average Test Recall: 0.1310732093387057

 Average Threshold F1 Score: 0.13235714904120804
Average Threshold Precision: 0.4778839839773711
Average Threshold Recall: 0.11568066060562987

In [ ]:
# safe XGBoost classifier
joblib.dump(model, '/content/drive/My Drive/ms_wind_curtailment_prediction/extra_trees_classifier.pkl')

['/content/drive/My Drive/ms_wind_curtailment_prediction/extra_trees_classifier.pkl']